<a href="https://colab.research.google.com/github/anastasiiaCher/educational-plans-evaluation/blob/main/test_embeds/embeddings_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/anastasiiaCher/educational-plans-evaluation.git

Cloning into 'educational-plans-evaluation'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 47 (delta 14), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (47/47), 4.02 MiB | 9.10 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [2]:
import sys
sys.path.append("/content/educational-plans-evaluation/test_embeds")

In [8]:
%%capture
!pip install pymorphy2 razdel

In [9]:
%%capture
import pandas as pd

import re
from operator import itemgetter
from random import randint
from time import time
import pymorphy2
from nltk import collocations
from nltk.corpus import stopwords
from razdel import sentenize, tokenize


import outcomes_extraction as ke
import difflib
from ast import literal_eval
from pprint import pprint
import numpy as np
from scipy.spatial.distance import cosine

import warnings
warnings.filterwarnings(action="ignore")

### Загрузка данных

In [10]:
!gdown 1DY-e32o-yIvDVvcGYOAfgT46cAYewDxw
!gdown 12bp0eeRdDt8C-UM_YB-URy-w2UvGtBDk
!gdown 1eMWo6bBQ5_hgYJsoYcxRR3gPGWAlDEMO

Downloading...
From: https://drive.google.com/uc?id=1DY-e32o-yIvDVvcGYOAfgT46cAYewDxw
To: /content/disc_contents20230506_4.csv
100% 70.6M/70.6M [00:01<00:00, 44.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=12bp0eeRdDt8C-UM_YB-URy-w2UvGtBDk
To: /content/dataprocessing_items_202305252034.csv
100% 1.45M/1.45M [00:00<00:00, 40.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1eMWo6bBQ5_hgYJsoYcxRR3gPGWAlDEMO
To: /content/clusters_of_disciplines.csv
100% 970k/970k [00:00<00:00, 33.3MB/s]


In [11]:
# сущности из БД
items = pd.read_csv("dataprocessing_items_202305252034.csv")
items["lowercase"] = items.name.str.lower()

# данные по дисциплинам из БД
df = pd.read_csv("disc_contents20230506_4.csv")

# кластеры сущностей
clusters = pd.read_csv("clusters_of_disciplines.csv")
clusters = clusters["nodes"].to_dict()

### Обработка данных

In [12]:
def get_entities(text, entities_to_match=150):
    """Здесь сущности извлекаются из текста и памятся с сущностями из БД
    150 - это вообще слишком много, но так сущности берутся по максимуму,
    отсюда примерно 3-6 секунд на выполнение извлечения
    Дальше то, что извлеклось, мапится с БД без ограничений по количеству"""

    example = ke.simple_outcomes_extraction(text, n_best=entities_to_match)
    res = pd.DataFrame(columns=["id", "name"])
    for word in example:
        close = difflib.get_close_matches(word, items.lowercase.tolist(), cutoff=0.85, n=1)
        if close:
            res = res.append(items[items.lowercase.isin(close)][["id", "name"]])
            res.drop_duplicates(subset=["name"], inplace=True)
    return res.name.tolist()



class Discipline:

    """Векторизация дисциплины по набору сущностей"""

    def __init__(self, entities, clusters):
        self.entities = entities
        self.n_entities = len(self.entities)
        self.clusters = clusters


    @property
    def vectorized_discipline(self):
        """векторизация дисциплины"""
        vector = np.zeros(len(self.clusters))
        for ent in self.entities:
            for cluster, values in self.clusters.items():
                if ent in values:
                    vector[cluster]+=1
        return vector


    @property
    def norm_vector(self):
        """нормализация вектора"""
        x = self.vectorized_discipline
        return (x-np.min(x))/(np.max(x)-np.min(x))


    @property
    def n_domains(self):
        return np.count_nonzero(self.vectorized_discipline)



### Тестирование

В датафрейме `df` можно найти по столбцу `id` данные дисциплины в Конструкторе. Значение для `id` берется из ссылки. В примере: https://op.itmo.ru/work-program/2856/general. В столбце `comb_res` хранятся учебные сущности дисциплины в виде списка.

In [13]:
discipline = df.query("id == 2856").iloc[0]
print(discipline.title)
entities = literal_eval(discipline.comb_res)
entities

Web-программирование


['Современные технологии разработки',
 'Разработка серверной части приложения',
 'Разработка интерфейсов web-приложений',
 'Разработка Single Page Application и Rich Internet Application',
 'vue.js',
 'django web framework',
 'django REST framework',
 'Сетевая модель OSI',
 'Адресация в сетях IP',
 'Модели данных',
 'Работа с токенами',
 'Паттерны проектирования']

Если дисциплины в файлике нет или это сгенеренный текст, то его можно его прямо текстом с разметкой передать в функцию `get_entities`.

In [14]:
TEXT = """- Основы Web-программирования
    - Основы HTML/CSS
        - Структура веб-страницы
        - Основы стилизации
    - JavaScript и фреймворки
        - Основы программирования на JavaScript
        - Введение в фреймворки JavaScript

- Разработка Backend и Базы Данных
    - Работа с базами данных
        - Основы SQL
        - Принципы проектирования баз данных
    - Backend разработка
        - Основы серверной логики
        - Программирование на Node.js

- Дизайн и Пользовательский Интерфейс
    - Принципы дизайна и UX/UI
        - Основы дизайна веб-интерфейсов
        - Основы UX/UI

- Тестирование и Безопасность
    - Тестирование и отладка
        - Методы тестирования веб-приложений
        - Инструменты отладки
    - Безопасность веб-приложений
        - Основы кибербезопасности
        - Защита от веб-угроз

- Продвинутые Темы
    - Разработка мобильных приложений
        - Основы мобильной веб-разработки
        - Адаптивный дизайн
    - Инструменты разработки и среды программирования
        - Работа с интегрированными средами разработки (IDE)
        - Версионный контроль
    - Разработка API
        - Принципы RESTful API
        - GraphQL
    - Работа с версионным контролем (Git)
        - Основы Git
        - Работа с ветками и слияниями
"""


entities_gen = get_entities(TEXT)
entities_gen

['Основы программирования',
 'Базы данных',
 'Веб-приложение',
 'Принципы проектирования ПО',
 'Методики тестирования',
 'Основы кибербезопасности',
 'Разработка мобильных приложений',
 'Разработка ПО',
 'Мобильное приложение',
 'Пользовательский интерфейс',
 'Проектирование БД',
 'Программирование',
 'Программирование роботов',
 'Кибербезопасность',
 'Основы дизайна',
 'Среда разработки VBA',
 'RESTful APIs',
 'Фреймворки',
 'веб-страница',
 'Веб-страница',
 'Данные',
 'Адаптивный дизайн',
 'Проектирование баз данных',
 '\ufeffВеб-разработка']

### Сравнение эмбеддингов

In [15]:
# тестируем векторизацию
print(f"Дисциплина: {discipline['title']}")

subj = Discipline(entities, clusters)

print(f"Количество сущностей у дисциплины: {subj.n_entities}")
pprint(subj.entities)

print(f"Количество предметных областей: {subj.n_domains}")
print(subj.vectorized_discipline)

print("Нормализованный вектор")
print(subj.norm_vector)

Дисциплина: Web-программирование
Количество сущностей у дисциплины: 12
['Современные технологии разработки',
 'Разработка серверной части приложения',
 'Разработка интерфейсов web-приложений',
 'Разработка Single Page Application и Rich Internet Application',
 'vue.js',
 'django web framework',
 'django REST framework',
 'Сетевая модель OSI',
 'Адресация в сетях IP',
 'Модели данных',
 'Работа с токенами',
 'Паттерны проектирования']
Количество предметных областей: 5
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 2. 8. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
Нормализованный вектор
[0.    0.    0.    0.    0.    0.    0.    0.125 0.    0.    0.    0.
 0.    0.25  1.    0.    0.    0.125 0.    0.    0.    0.    0.  

In [16]:
# сравнение двух дисциплин
subj2 = Discipline(entities_gen, clusters)
print(np.round(1-cosine(subj2.norm_vector, subj.norm_vector), 3))


0.618
